In [1]:
# ========== Imports ========== #
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

# ========== Configuration ========== #
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TEST_ROOT = 'C:\\Users\\mrinmoy\\Desktop\\Comys_Hackathon5\\Comys_Hackathon5\\Task_B\\train'  # ✅ UPDATE to your dataset path
IMG_SIZE = 224
EMBED_DIM = 256
N_WAY = 5
N_SHOT = 5
N_QUERY = 1
TEST_EPISODES = 250
MODEL_PATH = 'best_protonet.pth'  # ✅ Path to your saved model

# ========== Image Transform ========== #
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ========== Dataset Class ========== #
class FewShotDataset:
    def __init__(self, root_dir, transform, n_shot=5, n_query=1):
        self.root_dir = root_dir
        self.transform = transform
        self.n_shot = n_shot
        self.n_query = n_query

        self.classes = []
        self.class_to_images = {}

        for cls in os.listdir(root_dir):
            class_path = os.path.join(root_dir, cls)
            if not os.path.isdir(class_path):
                continue

            clean_images = [os.path.join(class_path, img) for img in os.listdir(class_path)
                            if img.endswith('.jpg') and not os.path.isdir(os.path.join(class_path, img))]

            distortion_path = os.path.join(class_path, 'distortion')
            if os.path.exists(distortion_path):
                distorted_images = [os.path.join(distortion_path, img) for img in os.listdir(distortion_path)
                                    if img.endswith('.jpg')]
            else:
                distorted_images = []

            if len(clean_images) >= self.n_query and len(distorted_images) >= self.n_shot:
                self.classes.append(cls)
                self.class_to_images[cls] = {'clean': clean_images, 'distorted': distorted_images}

        if len(self.classes) == 0:
            raise Exception("No valid classes found. Please check your dataset structure or sampling parameters.")

    def sample_episode(self, n_way=5):
        if len(self.classes) < n_way:
            raise Exception(f"Not enough classes available. Requested: {n_way}, Available: {len(self.classes)}")

        while True:
            selected_classes = random.sample(self.classes, n_way)

            support_images = []
            support_labels = []
            query_images = []
            query_labels = []

            label_mapping = {cls: idx for idx, cls in enumerate(selected_classes)}
            valid_episode = True

            for cls in selected_classes:
                clean_imgs = self.class_to_images[cls]['clean']
                distorted_imgs = self.class_to_images[cls]['distorted']

                if len(clean_imgs) < self.n_query or len(distorted_imgs) < self.n_shot:
                    valid_episode = False
                    break

                support_samples = random.sample(distorted_imgs, self.n_shot)
                query_samples = random.sample(clean_imgs, self.n_query)

                for img_path in support_samples:
                    img = Image.open(img_path).convert('RGB')
                    support_images.append(self.transform(img))
                    support_labels.append(label_mapping[cls])

                for img_path in query_samples:
                    img = Image.open(img_path).convert('RGB')
                    query_images.append(self.transform(img))
                    query_labels.append(label_mapping[cls])

            if valid_episode and support_images and query_images:
                return (torch.stack(support_images), torch.tensor(support_labels),
                        torch.stack(query_images), torch.tensor(query_labels), label_mapping)

# ========== Model ========== #
class ProtoNet(nn.Module):
    def __init__(self, embed_dim=EMBED_DIM):
        super().__init__()
        base = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(base.children())[:-1])
        self.fc = nn.Linear(base.fc.in_features, embed_dim)

    def forward(self, x):
        x = self.features(x).squeeze()
        return self.fc(x)

# ========== Euclidean Distance and Prediction ========== #
def euclidean_dist(a, b):
    n = a.size(0)
    m = b.size(0)
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    return torch.pow(a - b, 2).sum(2)

def predict_protonet(support_embeddings, support_labels, query_embeddings, n_way):
    prototypes = []
    for c in range(n_way):
        class_mask = (support_labels == c)
        class_embeddings = support_embeddings[class_mask]
        class_proto = class_embeddings.mean(0)
        prototypes.append(class_proto)
    prototypes = torch.stack(prototypes)
    dists = euclidean_dist(query_embeddings, prototypes)
    scores = -dists
    return scores

# ========== Test Script ========== #
def test_protonet(model, test_dataset, device, n_way=5, n_shot=5, n_query=1, episodes=250):
    model.eval()
    all_preds = []
    all_labels = []

    for _ in tqdm(range(episodes)):
        support_x, support_y, query_x, query_y, label_mapping = test_dataset.sample_episode(n_way)

        idx_to_class = {v: k for k, v in label_mapping.items()}

        support_x, support_y = support_x.to(device), support_y.to(device)
        query_x, query_y = query_x.to(device), query_y.to(device)

        with torch.no_grad():
            support_emb = model(support_x)
            query_emb = model(query_x)
            logits = predict_protonet(support_emb, support_y, query_emb, n_way)
            probs = F.softmax(logits, dim=1)
            preds = logits.argmax(1).cpu().numpy()
            true_labels = query_y.cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(true_labels)

        probs = probs.cpu().numpy()
        for i in range(len(preds)):
            true_class = idx_to_class[true_labels[i]]
            pred_class = idx_to_class[preds[i]]
            confidence_score = probs[i][preds[i]]
            match = 1 if preds[i] == true_labels[i] else 0

            print(f"Real: {true_class} | Pred: {pred_class} | Score: {confidence_score:.3f} | Match: {match}")

    # Overall Metrics
    top1_acc = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"\nTest Results:")
    print(f"Top-1 Accuracy: {top1_acc:.4f}")
    print(f"Macro-averaged F1-Score: {macro_f1:.4f}")

# ========== Main Execution ========== #
if __name__ == "__main__":
    # Load model
    model = ProtoNet().to(DEVICE)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))

    # Load test dataset
    test_dataset = FewShotDataset(TEST_ROOT, transform, n_shot=N_SHOT, n_query=N_QUERY)

    # Run test
    test_protonet(model, test_dataset, DEVICE, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, episodes=TEST_EPISODES)


c:\Users\mrinmoy\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mrinmoy\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 1/250 [00:02<12:23,  2.99s/it]

Real: Cabas | Pred: Cabas | Score: 0.962 | Match: 1
Real: Michael_Donovan | Pred: Michael_Donovan | Score: 1.000 | Match: 1
Real: Christine_Arron | Pred: Christine_Arron | Score: 0.937 | Match: 1
Real: Marwan_Barghouthi | Pred: Marwan_Barghouthi | Score: 0.726 | Match: 1
Real: Sophia_Loren | Pred: Marwan_Barghouthi | Score: 0.597 | Match: 0


  1%|          | 2/250 [00:05<11:07,  2.69s/it]

Real: Ali_Khamenei | Pred: Ali_Khamenei | Score: 0.895 | Match: 1
Real: Conchita_Martinez | Pred: Conchita_Martinez | Score: 0.843 | Match: 1
Real: Max_von_Sydow | Pred: Max_von_Sydow | Score: 0.878 | Match: 1
Real: Marcelo_Ebrard | Pred: Marcelo_Ebrard | Score: 0.485 | Match: 1
Real: Ruben_Studdard | Pred: Ruben_Studdard | Score: 0.990 | Match: 1


  1%|          | 3/250 [00:07<10:31,  2.56s/it]

...

Real: Yasushi_Akashi | Pred: Yasushi_Akashi | Score: 0.614 | Match: 1
Real: Mark_Andrew | Pred: Mark_Andrew | Score: 0.955 | Match: 1
Real: Marcos_Daniel_Jimenez | Pred: Marcos_Daniel_Jimenez | Score: 0.990 | Match: 1
Real: Wang_Nan | Pred: Wang_Nan | Score: 1.000 | Match: 1
Real: Kurt_Hellstrom | Pred: Kurt_Hellstrom | Score: 0.879 | Match: 1


100%|█████████▉| 249/250 [14:22<00:03,  3.32s/it]

Real: Richard_Rodriguez | Pred: Richard_Rodriguez | Score: 0.754 | Match: 1
Real: Mikulas_Dzurinda | Pred: Mikulas_Dzurinda | Score: 0.897 | Match: 1
Real: Sureyya_Ayhan | Pred: Sureyya_Ayhan | Score: 0.820 | Match: 1
Real: Joan_Dangerfield | Pred: Joan_Dangerfield | Score: 0.982 | Match: 1
Real: Doug_Moe | Pred: Doug_Moe | Score: 0.719 | Match: 1


100%|██████████| 250/250 [14:26<00:00,  3.47s/it]

Real: 088_frontal | Pred: 088_frontal | Score: 1.000 | Match: 1
Real: Tammy_Helm | Pred: Tammy_Helm | Score: 0.769 | Match: 1
Real: Alan_Tang_Kwong-wing | Pred: Alan_Tang_Kwong-wing | Score: 0.959 | Match: 1
Real: Chin-Feng_Chen | Pred: Chin-Feng_Chen | Score: 0.936 | Match: 1
Real: Shannon_OBrien | Pred: Shannon_OBrien | Score: 0.489 | Match: 1

Test Results:
Top-1 Accuracy: 0.8616
Macro-averaged F1-Score: 0.8616
